In [ ]:

import yaml
from train_eval.trainer import Trainer
import torch
import torch.nn as nn
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "1"


In [ ]:
test= torch.zeros(4,3,5,5)
test[0,0,1,1]=1
test.to_sparse()


In [ ]:

kernel_size=3
channel=2
unfold = nn.Unfold(kernel_size=(kernel_size, kernel_size), dilation=1, padding=kernel_size//2, stride=(1, 1))


In [ ]:
a=unfold(test).permute(0,2,1)
b=a.view(a.shape[0],a.shape[1],channel,kernel_size**2)
for i,j in enumerate(a):
    print(j)

In [ ]:
def get_unfolded_feature(feature,kernel,mask):
    ## Input shape B,C,H,W
    channel=feature.shape[1]
    unfold = nn.Unfold(kernel_size=(kernel, kernel), dilation=1, padding=kernel//2, stride=(1, 1))
    unfolded_feature=unfold(feature).permute(0,2,1) ## B,Number of slided window, channel*Number of elements in every window
    # unfolded_feature=unfolded_feature.view(unfolded_feature.shape[0],unfolded_feature.shape[1],channel,kernel**2)
    current_node_feat=feature.view(feature.shape[0],channel,-1).permute(0,2,1)## B,Number of slided window, channel
    # unfolded_feature=(unfolded_feature*mask.unsqueeze(-1)).to_sparse()
    # current_node_feat=(current_node_feat*mask.unsqueeze(-1)).to_sparse()
    target_feat=[]
    source_feat=[]
    for idx,batch in enumerate(current_node_feat):
        source_feat.append(batch[mask[idx]])
        target_feat.append(unfolded_feature[idx][mask[idx]])
    return source_feat,target_feat
source_feat,target_feat=get_unfolded_feature(op,15,mask_map)

In [ ]:
from models.library.blocks import LayerNorm
deocde_block=nn.Sequential(
                nn.Linear(2*48+2,48),
                LayerNorm(48),
                nn.LeakyReLU(),
                nn.Linear(48,48//2),
                LayerNorm(48//2),
                nn.LeakyReLU(),
                nn.Linear(48//2,1)
            ).to(op.device)
softmax=nn.Softmax(dim=-1)
def concat_feat(source_feat,target_feat,diff,channel):
    diff=diff.unsqueeze(0).to(source_feat[0].device)
    for idx,batch in enumerate(source_feat):
        target_batch=target_feat[idx].view(target_feat[idx].shape[0],channel,-1).permute(0,2,1)
        batch=batch.unsqueeze(-1).repeat(1,1,target_batch.shape[-2]).permute(0,2,1)
        concat_feat=torch.cat((batch,target_batch,diff.repeat(batch.shape[0],1,1)),dim=-1)
        connectivity=softmax(deocde_block(concat_feat).squeeze(-1))
concat_feat(source_feat,target_feat,diff,48)

In [ ]:
x_coord,y_coord=torch.meshgrid(torch.arange(15//2,-((15//2)+1),-1),
                                torch.arange(-(15//2),((15//2)+1),1))
diff=torch.cat([x_coord.unsqueeze(0),y_coord.unsqueeze(0)],dim=0)
diff=diff.view(2,-1).T
diff

In [ ]:
with open("/home/stanliu/code/pgp/PGP/configs/raster.yml", 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)

In [ ]:
ds_type = cfg['dataset'] + '_' + cfg['agent_setting'] + '_' + cfg['input_representation']
# cfg
# cfg['encoder_args']
# cfg.keys()

In [ ]:
trainer = Trainer(cfg, "/home/stanliu/data/mnt/nuScenes/", "/home/stanliu/code/pgp/PGP/preprocess_raster")

In [ ]:
for i,data in enumerate(trainer.tr_dl):
    data_test=data['inputs']
    gt_test=data['ground_truth']
    break


In [ ]:
focal=trainer.losses[0]
# dri_loss=trainer.losses[1]


In [ ]:
# data['ground_truth']['traj']
map_representation = data_test['map_representation'][0]
mask=data_test['map_representation'][1]


mask.shape

In [ ]:
import torch.nn as nn
unfold = nn.Unfold(kernel_size=(7, 7), dilation=1, padding=7//2, stride=(1, 1))
unfold(mask.unsqueeze(1)).shape

In [ ]:
from models.library.RasterSampler import *
sampler = Sampler(cfg['aggregator_args'],resolution=1.0,apply_mask=True)
nodes_2D=sampler.sample_goals(mask)
mask_under=sampler.sample_mask(mask)

In [ ]:
import torch.nn as nn
query_emb = nn.Linear(2, 32).to('cuda:0')
for batch in nodes_2D:
    node_emb=query_emb(batch)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(255*test_htmap[0,:3].permute(1,2,0)*mask)
plt.show()
plt.imshow(x)
plt.show()
plt.imshow(map_representation[0].permute(1,2,0))
plt.show()
plt.imshow(mask.squeeze(-1))
plt.show()

In [ ]:
from math import exp
from torch.autograd import Variable
import torch.nn.functional as F
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()
def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window
window=create_window(7, 12)
gaussian(7, 1)
test_heatmap=F.conv2d(test_htmap, window, padding = 7//2, groups = 12)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(data_test['surrounding_agent_representation'][4,:3].permute(1,2,0))
plt.show()
# plt.imshow(test_heatmap[0,0])
# plt.show()
# data['ground_truth']['traj']

In [ ]:
encodings=trainer.model.encoder(data_test)


In [ ]:
import time
target_agent_enc = encodings['target_agent_encoding']
context_enc = encodings['context_encoding']
if context_enc['combined'] is not None:
    combined_enc, map_mask = context_enc['combined'], context_enc['map_masks'].bool()
dim_reduction_block=nn.Sequential(
            nn.Conv2d(544, 528, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(528),
            nn.ReLU(),
            nn.Conv2d(528, 512, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU()
        ).to('cuda:0')
query_emb = nn.Linear(2, 128).to('cuda:0')
key_emb = nn.Linear(512, 128).to('cuda:0')
val_emb = nn.Linear(512, 128).to('cuda:0')
mha = nn.MultiheadAttention(128, 2).to('cuda:0')
augmented_target_agent_enc = target_agent_enc.unsqueeze(2).unsqueeze(3).repeat(1,1,combined_enc.shape[-2],combined_enc.shape[-1])
concatenated_encodings=torch.cat([combined_enc,augmented_target_agent_enc],dim=1)
context_encoding = dim_reduction_block(concatenated_encodings)##Fuse agent feat with map feat by compressing the dimension (actually is linear layer)
context_encoding = context_encoding.view(context_encoding.shape[0], context_encoding.shape[1], -1)## [Batch number, channel, H*W]
context_encoding = context_encoding.permute(0, 2, 1)## [Batch number, H*W, channel]
from models.library.RasterSampler import *
t0=time.time()
sampler = Sampler(cfg['aggregator_args'],resolution=1.0,apply_mask=True)
nodes_2D=sampler.sample_goals(map_mask)
mask_under=sampler.sample_mask(map_mask)

nodes_2D=pad_tensor(nodes_2D)
        
query = query_emb(nodes_2D).permute(1,0,2)
keys = key_emb(context_encoding).permute(1, 0, 2)
vals = val_emb(context_encoding).permute(1, 0, 2)
attn_output, _ = mha(query, keys, vals)
time.time()-t0

In [ ]:
import torch
def pad_tensor(node_list):
    max_num = max([batch.size(0) for batch in node_list])
    image_batch = []
    for img in node_list:
        image_batch.append(torch.cat((img, torch.zeros(max_num - img.size(0),img.size(1),device='cuda:0')), 0).unsqueeze(0))
    return torch.cat(image_batch,dim=0).to('cuda:0')

# pad_tensor(nodes_2D).shape

In [ ]:
def get_init(compensation,H,W):
    compensation=compensation.long()
    initial_pos=torch.zeros([H,W]).to(device)
    initial_pos[compensation[0],compensation[1]]=1
    return initial_pos.view(-1)
compensate=torch.Tensor([100,61])
init_pos=get_init(compensate,122,122)
init_states=[]
for batch in range(attn_output_weights.shape[0]):
    init_states.append(init_pos[mask_under[batch]].unsqueeze(1))

In [ ]:
for batch in init_states:
    print(pad_tensor(batch).shape) 

In [ ]:
target_agent_enc = encodings['target_agent_encoding']
context_enc = encodings['context_encoding']
if context_enc['combined'] is not None:
    combined_enc, map_mask = context_enc['combined'], context_enc['map_masks'].bool()
dim_reduction_block=nn.Sequential(
            nn.Conv2d(544, 528, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(528),
            nn.ReLU(),
            nn.Conv2d(528, 512, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU()
        ).to('cuda:0')
query_emb = nn.Linear(2, 128).to('cuda:0')
key_emb = nn.Linear(512, 128).to('cuda:0')
val_emb = nn.Linear(512, 128).to('cuda:0')
mha = nn.MultiheadAttention(128, 2).to('cuda:0')
augmented_target_agent_enc = target_agent_enc.unsqueeze(2).unsqueeze(3).repeat(1,1,combined_enc.shape[-2],combined_enc.shape[-1])
concatenated_encodings=torch.cat([combined_enc,augmented_target_agent_enc],dim=1)
context_encoding = dim_reduction_block(concatenated_encodings)##Fuse agent feat with map feat by compressing the dimension (actually is linear layer)
context_encoding = context_encoding.view(context_encoding.shape[0], context_encoding.shape[1], -1)## [Batch number, channel, H*W]
context_encoding = context_encoding.permute(0, 2, 1)## [Batch number, H*W, channel]
from models.library.RasterSampler import *
sampler = Sampler(cfg['aggregator_args'],resolution=1.0)
nodes_2D=sampler.sample_goals().repeat(context_encoding.shape[0],1,1).type(torch.float32)
t1=time.time()
mask_under=sampler.sample_mask(map_mask)
query = query_emb(nodes_2D).permute(1,0,2)
keys = key_emb(context_encoding).permute(1, 0, 2)
vals = val_emb(context_encoding).permute(1, 0, 2)
attn_output, attn_output_weights = mha(query, keys, vals)
time.time()-t1



In [ ]:

agg_feat=trainer.model.aggregator(encodings)
op=agg_feat['agg_encoding']
mask_map=agg_feat['under_sampled_mask']
print(op.requires_grad)
print(mask_map.requires_grad)

In [ ]:
predictions=trainer.model.decoder(agg_feat)

In [ ]:
attn_output_weights = agg_feat['node_connectivity']
init_states=agg_feat['initial_states']
predictions=torch.empty([init_states.shape[0],0,init_states.shape[-1]],device=attn_output_weights.device)
prev_states=init_states.unsqueeze(1)

for step in range(12):
    predictions=torch.cat((predictions,torch.bmm(prev_states,attn_output_weights)),dim=1)
    prev_states=predictions[:,step].unsqueeze(1)

In [ ]:
pred=predictions['pred']
mask=predictions['mask']

In [ ]:
def get_index(pred,mask):
    x_coord,y_coord=torch.meshgrid(torch.arange(0,122,1),
                                torch.arange(0,122,1))
    nodes_candidates=torch.cat((x_coord.unsqueeze(0),y_coord.unsqueeze(0)),dim=0).view(2,-1).T
    nodes_2D=torch.zeros([mask.shape[0],pred.shape[-1],2])
    for i in range(mask.shape[0]):
        nodes_batch=nodes_candidates[mask[i]]
        nodes_2D[i,:nodes_batch.shape[0]]=nodes_batch
    return nodes_2D.int().permute(0,2,1).to(pred.device)
nodes_2D=get_index(pred,mask)
nodes_2D.shape

In [ ]:
def get_dense(pred,nodes_2D,H,W):
    dense_rep=torch.empty(0,pred.shape[1],H,W,device=pred.device)
    for batch in range(pred.shape[0]):
        batch_heatmap=torch.empty(0,H,W,device=pred.device)
        for step in range(pred.shape[1]):
            heatmap=torch.sparse_coo_tensor(nodes_2D[batch],pred[batch,step],(122,122))
            batch_heatmap=torch.cat((batch_heatmap,heatmap.to_dense().unsqueeze(0)),dim=0)
        dense_rep=torch.cat((dense_rep,batch_heatmap.unsqueeze(0)),dim=0)
    return dense_rep
dense_rep=get_dense(pred,nodes_2D,122,122)

In [ ]:




# loss=pred.sum()
loss=focal.compute(predictions,gt_test)
loss.backward()


In [ ]:
trainer.model.aggregator

In [ ]:
pred = predictions['pred']
mask_da = predictions['mask'].view(-1,pred.shape[-2],pred.shape[-1]).unsqueeze(1)
ground_truth = gt_test
traj_gt = ground_truth['traj'] if type(ground_truth) == dict else ground_truth
true_heatmap,gs_map = focal.generate_gtmap(traj_gt,pred.shape)
gs_map=gs_map*mask_da
mask = (true_heatmap == 1).float()
pred_heatmap = torch.clamp(pred, min=1e-4)

In [ ]:
-torch.sum(
                torch.pow(pred_heatmap - gs_map, 2) * (
                mask * torch.log(pred_heatmap)
                +
                (1-mask) * (torch.pow(1 - gs_map, 4) * torch.log(1 - pred_heatmap))
            )
        )

In [ ]:
dri_loss.compute(predictions,gt_test)

In [ ]:
loss.backward()

In [ ]:
pred = predictions['pred']
mask = predictions['mask'].view(-1,pred.shape[-2],pred.shape[-1]).unsqueeze(1)
non_drivable_area_mask=~mask

In [ ]:
non_drivable_area_mask[0,0,0].cpu()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(non_drivable_area_mask[0,0].cpu())
plt.show()
plt.imshow(torch.zeros_like(non_drivable_area_mask[0,0].cpu()))
plt.show()

In [ ]:
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.nuscenes import NuScenes
map_extent=[ -50, 50, -20, 80 ]
img_size=[400,400]
resolution = (map_extent[1] - map_extent[0]) /  img_size[1]
nusc = NuScenes(version='v1.0-trainval', dataroot="/home/stanliu/data/mnt/nuScenes/", verbose=True)
helper=PredictHelper(nusc)
map_rasterizer = StaticLayerRasterizer(helper,
                                        resolution=resolution,
                                        meters_ahead=map_extent[3],
                                        meters_behind=-map_extent[2],
                                        meters_left=-map_extent[0],
                                        meters_right=map_extent[1])

In [ ]:
import torch
target_agent_representation = data_test['target_agent_representation']
surrounding_agent_representation = data_test['surrounding_agent_representation']
map_representation = data_test['map_representation'][0]
mask= data_test['map_representation'][1].type(torch.bool)
mask.shape

In [ ]:
idx=10
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(np.array(mask[idx]))
plt.show()
plt.imshow(np.array(map_representation[idx]).transpose(1,2,0))

In [ ]:

from torchvision.models import resnet34
# input = torch.cat((map_representation, surrounding_agent_representation), dim=1)
# resnet_model = resnet34(pretrained=False)
# conv1_new = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# modules = list(resnet_model.children())[:-2]

# modules[0] = conv1_new
# backbone = nn.Sequential(*modules)
# data_test['target_agent_representation'].float().dtype

In [ ]:
from models.encoders.raster_encoder import *
encoder=RasterEncoder(cfg['encoder_args'])
# encodings=encoder.forward(data_test)

In [ ]:
encodings['context_encoding']["combined"].shape

In [ ]:
import torch
import torch.nn.modules as nn
fake_map_encodings=torch.randn(32, 512, 16,16)
fake_agent_input=torch.randn(32, 32).unsqueeze(2).unsqueeze(3).repeat(1,1,16,16)

In [ ]:
concatenated_encodings=torch.cat([fake_map_encodings,fake_agent_input],dim=1)
conv1d1=nn.Conv2d(544, 528, kernel_size=1, stride=1, bias=False)
conv1d2=nn.Conv2d(528, 512, kernel_size=1, stride=1, bias=False)
test_dim_reduction=nn.Sequential(conv1d1,nn.BatchNorm2d(528),nn.ReLU(),conv1d2,nn.BatchNorm2d(512),nn.ReLU())
fake_feature=test_dim_reduction(concatenated_encodings)
fake_feature=fake_feature.view(fake_feature.shape[0], fake_feature.shape[1], -1).permute(0, 2, 1)

fake_feature.shape

In [ ]:
upsampled_test=final_convs(transpose_convs(fake_feature))
# augmented_mask=mask.unsqueeze(-1)
print(upsampled_test[:,:,::2,::2].shape)


In [ ]:
import numpy as np


import torch
from torch import nn

# from model.decode import generic_decode

# from utils.image import gaussian_radius, draw_umich_gaussian

# from model.ConvGRU import ConvGRU

from einops import rearrange as rearr, repeat

from spatial_correlation_sampler import spatial_correlation_sample


class LocalWalk(nn.Module):
    def __init__(self, topk=0, radius=0.05, temp=0.05, pad_value=0,
            broadcast_val=False, corr_module=True):
        super(LocalWalk, self).__init__()

        self.topk = topk
        self.radius = radius
        self.vals = {}
        self.idxmaps = {}
        self.temp = temp
        self.pad_value = pad_value

        self.broadcast_val = broadcast_val

        self.corr_module = corr_module

    def get_identity_label(self, keys):
        '''
        returns 1 x H*W x H x W as reshaped H*W x H*W identity matrix
        '''
        B, C, H, W = keys.shape
        name = f"{H}_{W}"
        if name not in self.vals:
            vals = self.distance_field(H, W).flatten(0, 1)
            vals = (vals == 0).float() ##Returns an identity matrix, which is composed of multiple matrices.
            # The i th matrix has an element 1 at the i ith position, the rest places are all zeros.
            vals = repeat(vals, 'n h w -> b n h w', b=B if not self.broadcast_val else 1)## Repeat for batch number times
            self.vals[name] = vals.to(keys.device)
            print('created vals')

        return self.vals[name]

    def knn(self, A):
        if self.pad_value == 0 or self.topk > 0:
            with torch.no_grad():
                mask = (A == self.pad_value)
                if self.topk > 0:
                  mask |= (A < A.topk(k=self.topk, dim=-1)[0].min(-1, keepdim=True)[0])
            A[mask] = -10

        return A

    def distance_field(self, H, W, p=2):
        gx, gy = torch.meshgrid(torch.arange(0, H), torch.arange(0, W))
        D = ( (gx[None, None, :, :] - gx[:, :, None, None]).abs()**p + (gy[None, None, :, :] - gy[:, :, None, None]).abs()**p ).float() #** (1/p)
        return D

    def make_scatter_map(self, keys, kH, kW):
        B, C, H, W = keys.shape
        name = f"{H}_{W}_{kH}_{kW}"
        if name not in self.idxmaps:
            idx_map = torch.arange(H*W).view(H, W)[None, None].float()
            idx_map = torch.nn.functional.unfold(idx_map, kernel_size=(kH, kW), stride=1, padding=(kH//2, kW//2))
            idx_map = rearr(idx_map, 'b n hw -> b hw n')
            idx_map = idx_map.clamp(min=0).long()
            self.idxmaps[name] = idx_map.to(keys.device)
            print('created idx map')

        return self.idxmaps[name]

    def forward(self, query, keys, val=None):
        '''
        assumes q, k, v: B D N
        '''

        B, C, H, W = keys.shape
        kW = kH = int(H * self.radius) * 2 + 1

        val = self.make_scatter_map(keys, kH, kW)## Returns the indices of elements inside the sliding windows of all steps
        ## The sliding window has size (kH,kW), the input is keys.
        # out = self.get_identity_label(keys) * 0##Why all zeros?
        out = torch.zeros([B,H*W,H,W]).float()
        out = repeat(out, '1 n h w -> b (h w) n', b=B) if out.shape[0] == 1 else \
              rearr(out, 'b n h w -> b (h w) n') ##Repeat for batch number

        if self.corr_module:
            att = spatial_correlation_sample(query,
                               keys,
                               kernel_size=1,
                               patch_size=kH,
                               stride=1,
                               padding=0,
                               dilation=1,
                               dilation_patch=1) / self.temp
            att = rearr(att, 'b p1 p2 h w -> b h w (p1 p2)')##Local connectivity, for each node (pixel)
            ## calculate its node similarity with nearby nodes

        A = self.knn(att)

        A = torch.exp(rearr(A, 'b h w n -> b (h w) n'))
        out.scatter_add_(2, val.to(A.device).expand_as(A), A)
        val = rearr(out, 'b (h w) n -> b n h w', h=H)

        return val

In [ ]:
test_tensor=torch.randn([32,20,100,100])
test_walker=LocalWalk(radius=0.2)
test_walker.forward(test_tensor,test_tensor).shape

In [ ]:
from models.library.RasterSampler import *
sampler=Sampler(cfg['train_set_args'],1)

In [ ]:
fake_mask=torch.randn([32,488,488]).ge(0)
nodes_2D=sampler.sample_goals().repeat(32,1,1).type(torch.float32)
mask_under=(sampler.sample_mask(mask))
attn_mask=~mask_under.unsqueeze(-1).repeat(2,1,256)
print(attn_mask.shape)
print(mask.shape)
print(nodes_2D.shape)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.array(mask[0]))
plt.show()
plt.imshow(np.array(~mask_under[0].view(122,122)))
plt.show()

In [ ]:
fake_feature.shape
lin1=nn.Linear(2, 128)
lin2=nn.Linear(512, 128)
print(lin1(nodes_2D.type(torch.float32)).shape)
print(lin2(fake_feature.type(torch.float32)).shape)
print(attn_mask.shape)


In [ ]:
class Attention():
    """
    Test fusing 2D goal encodings and map encodings with attention mechanism. 
    """

    def __init__(self):

        """
        args to include

        enc_size: int Dimension of encodings generated by encoder
        emb_size: int Size of embeddings used for queries, keys and values
        num_heads: int Number of attention heads

        """
        self.query_emb = nn.Linear(2, 128)
        self.key_emb = nn.Linear(512, 128)
        self.val_emb = nn.Linear(512, 128)
        self.mha = nn.MultiheadAttention(128, 2)

    def forward(self, query_goals, map_features, attn_mask) -> torch.Tensor:
        """
        Forward pass for attention aggregator
        """

        query = self.query_emb(query_goals).permute(1,0,2)
        keys = self.key_emb(map_features).permute(1,0,2)
        vals = self.val_emb(map_features).permute(1,0,2)
        op, _ = self.mha(query, keys, vals, attn_mask=attn_mask)
        # op = op.squeeze(0)
        # op = torch.cat((target_agent_enc, op), dim=-1)

        return op

In [ ]:
from models.library.blocks import TransposeCNNBlock
trans_conv=nn.Sequential([
            TransposeCNNBlock(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=0),
            TransposeCNNBlock(in_channels=256, out_channels=128, kernel_size=3, stride=2, output_padding=0),
            TransposeCNNBlock(in_channels=128, out_channels=64, kernel_size=3, stride=2, output_padding=0),
            TransposeCNNBlock(in_channels=64, out_channels=32, kernel_size=3, stride=2, output_padding=1)
        ])